# A Diffusion Model from Scratch in Pytorch

In this notebook I want to build a very simple (as few code as possible) Diffusion Model for generating car images. I will explain all the theoretical details in the YouTube video.


**Sources:**
- Github implementation [Denoising Diffusion Pytorch](https://github.com/lucidrains/denoising-diffusion-pytorch)
- Niels Rogge, Kashif Rasul, [Huggingface notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/annotated_diffusion.ipynb#scrollTo=3a159023)
- Papers on Diffusion models ([Dhariwal, Nichol, 2021], [Ho et al., 2020] ect.)


## Investigating the dataset

As dataset we use the StandordCars Dataset, which consists of around 8000 images in the train set. Let's see if this is enough to get good results ;-)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
import scipy.io as sio
from time import time, sleep
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")  # adds seaborn style to charts, eg. grid
plt.style.use("dark_background")  # inverts colors to dark theme
plt.rcParams['font.family'] = 'monospace'
import os
import warnings; warnings.filterwarnings("ignore")
from utils import calc_gso_batch # gso/pinn calculation
try: 
    JOBID = os.environ["SLURM_JOB_ID"] # get job id from slurm, when training on cluster
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu") # nvidia
    HAS_SCREEN = False # for plotting or saving images
except:
    device = torch.device("mps") # apple silicon
    JOBID = "local"
    HAS_SCREEN = True
os.makedirs(f"mg_data/{JOBID}", exist_ok=True)
print(f'device: {device}')

# copy the python training to the directory (for cluster) (for local, it fails silently)
os.system(f"cp mg_train2.py mg_data/{JOBID}/mg_train2.py")
os.system(f"cp utils.py mg_data/{JOBID}/utils.py")

def to_tensor(x, device=torch.device("cpu")): return torch.tensor(x, dtype=torch.float32, device=device)

PRENAME_MSE, PRENAME_GSO, PRENAME_TOT = "mg_planet_mse", "mg_planet_gso", "mg_planet_tot"

In [ ]:
SAVE_DIR = f"mg_data/{JOBID}" 
EPOCHS = 1000 # number of epochs, note: needs to 
BATCH_SIZE = 128 # 128 best
IMG_SIZE = 64

LOAD_PRETRAINED = None # Set it to None if you don't want to load pretrained model

# LEARNING_RATE = 3e-4*np.linspace(1, 1e-2, EPOCHS)  # best
LEARNING_RATE = 3e-4*np.logspace(0, -2, EPOCHS)
# LEARNING_RATE = 1e-4*np.logspace(0, -2, EPOCHS)

# GSO_LOSS_RATIO = np.linspace(0.4, 0.1, EPOCHS) # best
# GSO_LOSS_RATIO = np.linspace(0.3, 0.1, EPOCHS) # best too
# GSO_LOSS_RATIO = np.linspace(0.4, 0.0, EPOCHS) # best for big model pretrain start
GSO_LOSS_RATIO = np.concatenate((np.linspace(0.4, 0.0, EPOCHS//2), np.linspace(0.0, 0.0, EPOCHS//2))) 
# GSO_LOSS_RATIO = 0.1*np.ones(EPOCHS) # not very good
# GSO_LOSS_RATIO = (0.5+0.5*np.sin(np.linspace(0, 25*np.pi, EPOCHS)))*np.linspace(1, 0.1, EPOCHS) # crazy

NCURRS, NPROFS, NMAGS = 14, 202, 187 # input sizes
INPUT_SIZE = NCURRS + NPROFS + NMAGS
TRAIN_DS_PATH = "data/train_ds_10000_80_100.mat" # generated from prepapre_dataset
EVAL_DS_PATH = "data/eval_ds_10000_80_100.mat"

In [ ]:
class PlaNetDataset(Dataset):
    def __init__(self, ds_mat_path, transforms=None):
        self.transforms = transforms
        ds_mat = sio.loadmat(ds_mat_path)
        # output: magnetic flux, transposed (matlab is column-major)
        self.psi = ds_mat["psi"]
        psi_mean, psi_std = np.mean(self.psi), np.std(self.psi) # mean and std for normalization
        self.psi = (to_tensor(self.psi)-psi_mean)/(5*psi_std)  # (n, 1, 64, 64) # normalized
        self.psi = self.psi.view(-1, 64, 64).cpu().numpy()
        self.psi = self.psi*255 + 127.5
        self.psi = self.psi.clip(0, 255)
        self.psi = self.psi.astype(np.uint8)
        # self.psi = to_tensor(ds_mat["psi"]).view(-1, 1, 64, 64)
        # inputs: radial and vertical position of pixels (for plotting only rn) + currents + measurements + profiles 
        self.rr = to_tensor(ds_mat["rr"]).view(-1,1,64,64) # radial position of pixels (64, 64)
        self.zz = to_tensor(ds_mat["zz"]).view(-1,1,64,64) # vertical position of pixels (64, 64)
        self.currs = ds_mat["currs"] # input currents (n, 14)
        self.mags = ds_mat["magnetic"] # input magnetic measurements (n, 187)
        f_prof = ds_mat["f_profiles"] # input profiles (n, 101)
        p_prof = ds_mat["p_profiles"] # input profiles (n, 101)
        # means and stdfs for normalization
        currs_mean, currs_std = np.mean(self.currs), np.std(self.currs)
        mags_mean, mags_std = np.mean(self.mags), np.std(self.mags)
        f_prof_mean, f_prof_std = np.mean(f_prof), np.std(f_prof)
        p_prof_mean, p_prof_std = np.mean(p_prof), np.std(p_prof)
        # normalize inputs
        self.currs = (to_tensor(self.currs)-currs_mean)/currs_std # (n, 14) # normalized
        self.mags = (to_tensor(self.mags)-mags_mean)/mags_std # (n, 187) # normalized
        self.profs = torch.cat(((to_tensor(f_prof)-f_prof_mean)/f_prof_std, (to_tensor(p_prof)-p_prof_mean)/p_prof_std), dim=1) # (n, 202) # normalized
        # move to device (doable bc the dataset is fairly small, check memory usage)
        # self.currs, self.mags, self.profs = self.currs.to(device), self.mags.to(device), self.profs.to(device)
        # self.psi, self.rr, self.zz = self.psi.to(device), self.rr.to(device), self.zz.to(device)
        self.everything = [self.currs, self.mags, self.profs, self.psi, self.rr, self.zz]
        # print(f"Dataset: {len(self)}, memory: {sum([x.element_size()*x.nelement() for x in self.everything])/1024**2:.0f} MB")
    def __len__(self): return len(self.psi)
    def __getitem__(self, idx): 
        x = self.psi[idx]
        # x = (x.view(64, 64)).cpu().numpy()
        # x = (x+1)*255/2
        # x = x.clip(0, 255)
        # x = x.astype(np.uint8)
        if self.transforms is not None: x = self.transforms(x)
        return x

In [ ]:
def show_images(data, num_samples=20, cols=4):
    """ Plots some samples from the dataset """
    plt.figure(figsize=(7,7))
    for i, img in enumerate(data):
        if i == num_samples: break
        plt.subplot(int(num_samples/cols) + 1, cols, i + 1)
        plt.grid(False)
        plt.xticks([])
        plt.yticks([])
        # plt.imshow(img[0])
        plt.imshow(img)

data = torchvision.datasets.StanfordCars(root="./", download=False)
print(f"Dataset: {len(data)}")
data = PlaNetDataset(TRAIN_DS_PATH)
print(f"Dataset: {len(data)}")
show_images(data)

Later in this notebook we will do some additional modifications to this dataset, for example make the images smaller, convert them to tensors ect.

# Building the Diffusion Model


## Step 1: The forward process = Noise scheduler




We first need to build the inputs for our model, which are more and more noisy images. Instead of doing this sequentially, we can use the closed form provided in the papers to calculate the image for any of the timesteps individually.

**Key Takeaways**:
- The noise-levels/variances can be pre-computed
- There are different types of variance schedules
- We can sample each timestep image independently (Sums of Gaussians is also Gaussian)
- No model is needed in this forward step

In [ ]:
import torch.nn.functional as F

def linear_beta_schedule(timesteps, start=0.0001, end=0.02):
    return torch.linspace(start, end, timesteps)

def get_index_from_list(vals, t, x_shape):
    """
    Returns a specific index t of a passed list of values vals
    while considering the batch dimension.
    """
    batch_size = t.shape[0]
    out = vals.gather(-1, t.cpu())
    return out.reshape(batch_size, *((1,) * (len(x_shape) - 1))).to(t.device)

def forward_diffusion_sample(x_0, t, device="cpu"):
    """
    Takes an image and a timestep as input and
    returns the noisy version of it
    """
    noise = torch.randn_like(x_0)
    sqrt_alphas_cumprod_t = get_index_from_list(sqrt_alphas_cumprod, t, x_0.shape)
    sqrt_one_minus_alphas_cumprod_t = get_index_from_list(
        sqrt_one_minus_alphas_cumprod, t, x_0.shape
    )
    # mean + variance
    return sqrt_alphas_cumprod_t.to(device) * x_0.to(device) \
        + sqrt_one_minus_alphas_cumprod_t.to(device) * noise.to(device), noise.to(device)


# Define beta schedule
T = 300
betas = linear_beta_schedule(timesteps=T)

# Pre-calculate different terms for closed form
alphas = 1. - betas
alphas_cumprod = torch.cumprod(alphas, axis=0)
alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)
sqrt_recip_alphas = torch.sqrt(1.0 / alphas)
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1. - alphas_cumprod)
posterior_variance = betas * (1. - alphas_cumprod_prev) / (1. - alphas_cumprod)

Let's test it on our dataset ...

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np

def load_transformed_dataset():
    data_transforms = [
        transforms.ToTensor(), # Scales data into [0,1]
        transforms.Lambda(lambda t: (t * 2) - 1) # Scale between [-1, 1]
    ]
    data_transform = transforms.Compose(data_transforms)
    return PlaNetDataset(TRAIN_DS_PATH, transforms=data_transform)
    
def show_tensor_image(image):
    reverse_transforms = transforms.Compose([
        transforms.Lambda(lambda t: (t + 1) / 2),
        transforms.Lambda(lambda t: t.permute(1, 2, 0)), # CHW to HWC
        transforms.Lambda(lambda t: t * 255.),
        transforms.Lambda(lambda t: t.numpy().astype(np.uint8)),
        transforms.ToPILImage(),
    ])

    # Take first image of batch
    if len(image.shape) == 4:
        image = image[0, :, :, :]
    plt.imshow(reverse_transforms(image), cmap="plasma")
    plt.axis("off")

data = load_transformed_dataset()
dataloader = DataLoader(data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

#print stats about dataloader
print(f"Number of batches: {len(dataloader)}")

In [ ]:
# Simulate forward diffusion
# image = next(iter(dataloader))[0]
image = next(iter(dataloader))[0]

plt.figure(figsize=(15,2))
plt.axis('off')
num_images = 12
stepsize = int(T/num_images)

for idx in range(0, T, stepsize):
    t = torch.Tensor([idx]).type(torch.int64)
    plt.subplot(1, num_images+1, int(idx/stepsize) + 1)
    img, noise = forward_diffusion_sample(image, t)
    show_tensor_image(img)

## Step 2: The backward process = U-Net



For a great introduction to UNets, have a look at this post: https://amaarora.github.io/2020/09/13/unet.html.


**Key Takeaways**:
- We use a simple form of a UNet for to predict the noise in the image
- The input is a noisy image, the ouput the noise in the image
- Because the parameters are shared accross time, we need to tell the network in which timestep we are
- The Timestep is encoded by the transformer Sinusoidal Embedding
- We output one single value (mean), because the variance is fixed


In [ ]:
from torch import nn
import math


class Block(nn.Module):
    def __init__(self, in_ch, out_ch, time_emb_dim, up=False):
        super().__init__()
        self.time_mlp =  nn.Linear(time_emb_dim, out_ch)
        if up:
            self.conv1 = nn.Conv2d(2*in_ch, out_ch, 3, padding=1)
            self.transform = nn.ConvTranspose2d(out_ch, out_ch, 4, 2, 1)
        else:
            self.conv1 = nn.Conv2d(in_ch, out_ch, 3, padding=1)
            self.transform = nn.Conv2d(out_ch, out_ch, 4, 2, 1)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1)
        self.bnorm1 = nn.BatchNorm2d(out_ch)
        self.bnorm2 = nn.BatchNorm2d(out_ch)
        self.relu  = nn.ReLU()

    def forward(self, x, t, ):
        # First Conv
        h = self.bnorm1(self.relu(self.conv1(x)))
        # Time embedding
        time_emb = self.relu(self.time_mlp(t))
        # Extend last 2 dimensions
        time_emb = time_emb[(..., ) + (None, ) * 2]
        # Add time channel
        h = h + time_emb
        # Second Conv
        h = self.bnorm2(self.relu(self.conv2(h)))
        # Down or Upsample
        return self.transform(h)


class SinusoidalPositionEmbeddings(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, time):
        device = time.device
        half_dim = self.dim // 2
        embeddings = math.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = time[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        # TODO: Double check the ordering here
        return embeddings


class SimpleUnet(nn.Module):
    """
    A simplified variant of the Unet architecture.
    """
    def __init__(self):
        super().__init__()
        image_channels = 1
        # down_channels = (64, 128, 256, 512, 1024)
        # up_channels = (1024, 512, 256, 128, 64)
        K = 8
        down_channels = (64//K, 128//K, 256//K, 512//K, 1024//K)
        up_channels = (1024//K, 512//K, 256//K, 128//K, 64//K)
        out_dim = 1
        # time_emb_dim = 32
        time_emb_dim = 16

        # Time embedding
        self.time_mlp = nn.Sequential(
                SinusoidalPositionEmbeddings(time_emb_dim),
                nn.Linear(time_emb_dim, time_emb_dim),
                nn.ReLU()
            )

        # Initial projection
        self.conv0 = nn.Conv2d(image_channels, down_channels[0], 3, padding=1)

        # Downsample
        self.downs = nn.ModuleList([Block(down_channels[i], down_channels[i+1], \
                                    time_emb_dim) \
                    for i in range(len(down_channels)-1)])
        # Upsample
        self.ups = nn.ModuleList([Block(up_channels[i], up_channels[i+1], \
                                        time_emb_dim, up=True) \
                    for i in range(len(up_channels)-1)])

        # Edit: Corrected a bug found by Jakub C (see YouTube comment)
        self.output = nn.Conv2d(up_channels[-1], out_dim, 1)

    def forward(self, x, timestep):
        # Embedd time
        t = self.time_mlp(timestep)
        # Initial conv
        x = self.conv0(x)
        # Unet
        residual_inputs = []
        for down in self.downs:
            x = down(x, t)
            residual_inputs.append(x)
        for up in self.ups:
            residual_x = residual_inputs.pop()
            # Add residual x as additional channels
            x = torch.cat((x, residual_x), dim=1)
            x = up(x, t)
        return self.output(x)

model = SimpleUnet()
print("Num params: ", sum(p.numel() for p in model.parameters()))
# model


**Further improvements that can be implemented:**
- Residual connections
- Different activation functions like SiLU, GWLU, ...
- BatchNormalization
- GroupNormalization
- Attention
- ...

## Step 3: The loss



**Key Takeaways:**
- After some maths we end up with a very simple loss function
- There are other possible choices like L2 loss ect.


In [ ]:
def get_loss(model, x_0, t):
    x_noisy, noise = forward_diffusion_sample(x_0, t, device)
    noise_pred = model(x_noisy, t)
    # return F.l1_loss(noise, noise_pred)
    return F.mse_loss(noise, noise_pred)

## Sampling
- Without adding @torch.no_grad() we quickly run out of memory, because pytorch tacks all the previous images for gradient calculation
- Because we pre-calculated the noise variances for the forward pass, we also have to use them when we sequentially perform the backward process

In [ ]:
@torch.no_grad()
def sample_timestep(x, t):
    """
    Calls the model to predict the noise in the image and returns
    the denoised image.
    Applies noise to this image, if we are not in the last step yet.
    """
    betas_t = get_index_from_list(betas, t, x.shape)
    sqrt_one_minus_alphas_cumprod_t = get_index_from_list(
        sqrt_one_minus_alphas_cumprod, t, x.shape
    )
    sqrt_recip_alphas_t = get_index_from_list(sqrt_recip_alphas, t, x.shape)

    # Call model (current image - noise prediction)
    model_mean = sqrt_recip_alphas_t * (
        x - betas_t * model(x, t) / sqrt_one_minus_alphas_cumprod_t
    )
    posterior_variance_t = get_index_from_list(posterior_variance, t, x.shape)

    if t == 0:
        # As pointed out by Luis Pereira (see YouTube comment)
        # The t's are offset from the t's in the paper
        return model_mean
    else:
        noise = torch.randn_like(x)
        return model_mean + torch.sqrt(posterior_variance_t) * noise

@torch.no_grad()
def sample_plot_image():
    # Sample noise
    img_size = IMG_SIZE
    img = torch.randn((1, 1, img_size, img_size), device=device)
    plt.figure(figsize=(15,2))
    plt.axis('off')
    num_images = 10
    stepsize = int(T/num_images)

    for i in range(0,T)[::-1]:
        t = torch.full((1,), i, device=device, dtype=torch.long)
        img = sample_timestep(img, t)
        # Edit: This is to maintain the natural range of the distribution
        img = torch.clamp(img, -1.0, 1.0)
        if i % stepsize == 0:
            plt.subplot(1, num_images, int(i/stepsize)+1)
            show_tensor_image(img.detach().cpu())
    plt.tight_layout()
    plt.show()

## Training

In [ ]:
from torch.optim import Adam
from tqdm import tqdm

data = load_transformed_dataset()
dataloader = DataLoader(data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

model.to(device)
optimizer = Adam(model.parameters(), lr=0.001)
epochs = 30 # Try more!

for epoch in range(epochs):
    for step, batch in enumerate(tqdm(dataloader, leave=False, desc=f"Epoch {epoch}")):
    # for step, batch in enumerate(dataloader):
        optimizer.zero_grad()

        t = torch.randint(0, T, (BATCH_SIZE,), device=device).long()
        loss = get_loss(model, batch, t)
        loss.backward()
        optimizer.step()

        if epoch % 5 == 0 and step == 0:
            print(f"\rEpoch {epoch} | Loss: {loss.item()} ")
            sample_plot_image()
    # save the model
    torch.save(model.state_dict(), f"mg_data/{JOBID}/diffusion_model.pth")

In [ ]:
# load the model
model = SimpleUnet()
model.load_state_dict(torch.load(f"mg_data/{JOBID}/diffusion_model.pth"))
model.to(device)

# sample and plot

@torch.no_grad()
def generate_samples(n, device="cpu"):
    imgs = torch.randn((n, 1, IMG_SIZE, IMG_SIZE), device=device) # Sample noise
    for i in range(0,T)[::-1]:
        t = torch.full((1,), i, device=device, dtype=torch.long)
        imgs = sample_timestep(imgs, t)
        # Edit: This is to maintain the natural range of the distribution
        imgs = torch.clamp(imgs, -1.0, 1.0)
    return imgs

# Generate samples
n = 7
samples = generate_samples(n*n, device)
plt.figure(figsize=(8, 8))
plt.axis('off')
for i in range(n):
    for j in range(n):
        plt.subplot(n, n, i*n+j+1)
        show_tensor_image(samples[i*n+j].detach().cpu())
plt.suptitle("Generated samples")
plt.tight_layout()
plt.show()

# samples from the dataset
data = load_transformed_dataset()
dataloader = DataLoader(data, batch_size=n*n, shuffle=True, drop_last=True)
# sample from the dataset
samples = next(iter(dataloader))
plt.figure(figsize=(8, 8))
plt.axis('off')
for i in range(n):
    for j in range(n):
        plt.subplot(n, n, i*n+j+1)
        show_tensor_image(samples[i*n+j].detach().cpu())
plt.suptitle("Real samples")
plt.tight_layout()
plt.show()